In [2]:
##!tar czf bert_zh_L-12_H-768_A-12_2.tar.gz -C ./戴月明的資料夾/bert_zh_L-12_H-768_A-12_2 .
!tar czf bert_zh_L-12_H-768_A-12_2.tar.gz -C  .

tar: Cowardly refusing to create an empty archive
Try 'tar --help' or 'tar --usage' for more information.


In [3]:
import tensorflow as tf 
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
import pandas as pd
from tqdm import tqdm

2021-09-13 09:01:44.783116: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0


In [4]:
df=pd.read_csv('NLP22355_all_clean.csv')
df.shape #(22355, 7)
df.tail(2)

,History,Schizophrenia,Psychotic_depression,Personality_disorders,ADJUST,mania,Substance_use_disorder
22353,根據個案本人敘述，個案成長及求學期間皆大致正常，否認有任何精神或物質濫用病史，其最高學歷為大...,0,1,0,0,0,0
22354,個案病前人格內向，少友，五專三年級肄業。在校成績中上，否認有在學期間有蹺課、打架遭記過之記錄...,1,0,0,0,0,0


In [5]:
df['Psychotic_depression'].value_counts()

1    12825
0     9530
Name: Psychotic_depression, dtype: int64

In [6]:
df['Dx']='7其他'
df.loc[df['ADJUST']==1, 'Dx']='6適應障礙'
df.loc[df['Substance_use_disorder']==1,'Dx']='5物質使用'
df.loc[df['Personality_disorders']==1, 'Dx']='4人格障礙'
df.loc[df['mania']==1, 'Dx']='3躁症'
df.loc[df['Psychotic_depression']==1,'Dx']='2憂鬱'
df.loc[df['Schizophrenia']==1,'Dx']='1思覺失調'
df.tail(10)

,History,Schizophrenia,Psychotic_depression,Personality_disorders,ADJUST,mania,Substance_use_disorder,Dx
22345,根據個案描述，父親從小管教嚴厲，最高學歷四海工專畢，72年結婚，婚後育有一女。約個案24歲時...,1,0,0,0,0,1,1思覺失調
22346,根據個案描述，父親從小管教嚴厲，最高學歷四海工專畢，72年結婚，婚後育有一女。約個案24歲時...,1,0,0,0,0,1,1思覺失調
22347,根據個案描述，父親從小管教嚴厲，最高學歷四海工專畢，72年結婚，婚後育有一女。約個案24歲時...,1,0,0,0,0,1,1思覺失調
22348,根據個案描述，父親從小管教嚴厲，最高學歷四海工專畢，72年結婚，婚後育有一女。約個案24歲時...,1,0,0,0,0,1,1思覺失調
22349,據個案及部隊表示，為緬甸華僑，來台以15年左右，病前個性外向，對讀書沒興趣，在學成績皆為倒數...,0,0,0,0,0,0,7其他
22350,個案約2年前來台灣取得身分證，之前在越南就有學中文，最高在越南的國中畢業。個案自述於103年...,0,1,0,1,0,0,2憂鬱
22351,個案自述於103年4月8日台南官田入伍，尚能適應。幾乎每天情緒悶，失去興趣喜樂(看書)，食慾...,0,1,0,1,0,0,2憂鬱
22352,根據個案自述，自小個性內向，求學期間成績中等，最高學歷為北台灣科技大學畢業(99年06月)。...,0,1,0,0,0,0,2憂鬱
22353,根據個案本人敘述，個案成長及求學期間皆大致正常，否認有任何精神或物質濫用病史，其最高學歷為大...,0,1,0,0,0,0,2憂鬱
22354,個案病前人格內向，少友，五專三年級肄業。在校成績中上，否認有在學期間有蹺課、打架遭記過之記錄...,1,0,0,0,0,0,1思覺失調


In [7]:
df=df[['History','Dx']]
df.shape
df.Dx.unique()
df['Dx'].value_counts()

2憂鬱      12384
1思覺失調     6097
7其他       1362
3躁症        939
4人格障礙      733
5物質使用      495
6適應障礙      345
Name: Dx, dtype: int64

In [8]:
df['y']=df['Dx'].str[0]
df['label']=df['Dx'].str[1:len(df['Dx'])]
df.tail(3)

,History,Dx,y,label
22352,根據個案自述，自小個性內向，求學期間成績中等，最高學歷為北台灣科技大學畢業(99年06月)。...,2憂鬱,2,憂鬱
22353,根據個案本人敘述，個案成長及求學期間皆大致正常，否認有任何精神或物質濫用病史，其最高學歷為大...,2憂鬱,2,憂鬱
22354,個案病前人格內向，少友，五專三年級肄業。在校成績中上，否認有在學期間有蹺課、打架遭記過之記錄...,1思覺失調,1,思覺失調


In [9]:
from transformers import TFBertForSequenceClassification, BertTokenizer
model = TFBertForSequenceClassification.from_pretrained("hfl/chinese-bert-wwm-ext",num_labels=7)
tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm-ext", from_tf=True)

2021-09-13 09:01:47.957672: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-09-13 09:01:47.965109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-13 09:01:47.965396: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-09-13 09:01:47.965424: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
2021-09-13 09:01:47.967710: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.11
2021-09-13 09:01:47.968945: I tensorflow/stream_executor/platform/default/d

ResourceExhaustedError: OOM when allocating tensor with shape[3072,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()

In [ ]:
text=df['History']
y=df['y']
tokenized_text=[tokenizer(i)['input_ids'] for i in text]
type(tokenized_text)

In [ ]:
tokenized_df=pd.DataFrame(
{'History':tokenized_text,'y':lb.fit_transform(y)})
tokenized_df.shape

In [ ]:
#檢查['History']是否有'NaN'值...
tokenized_df.tail(2)
NaNrow=tokenized_df.loc[tokenized_df['History']=='NaN']
NaNrow.shape

In [ ]:
##檢查字串長度
tokenized_df['length']=tokenized_df['History'].apply(len)
tokenized_df.tail(3)

In [ ]:
max_len=max(tokenized_df['length']) ##4912
import numpy as np
history_with_len=[[x,tokenized_df['y'], len(x)] for i,x in enumerate(tokenized_df['History'])]

In [ ]:
history_with_len.sort(key=lambda x: x[2])
sorted_history_labels=[(history_label[0],history_label[1]) for history_label in history_with_len]

In [ ]:
processed_dataset=tf.data.Dataset.from_generator(lambda:sorted_history_labels, output_types=(tf.int32,tf.int32))

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)


In [ ]:
max_len = tokenized_df['History'].str.len().max()

In [ ]:
[tf.data.Dataset.from_tensor_slices(([tokenized_df['History'][x]],[tokenized_df['y'][x]])) for i,x in enumerate(tokenized_df['History'])]